# **DeepRhythm**
Here I will try to implement a model that will predict the next note duration given a sequence of note-duration pairs.

In [ ]:
#Mounts google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
with open('/content/gdrive/My Drive/Google Colab/MusicGen/Data/Mozart/net_in_CLBM_L10E_160', 'rb') as filepath:
  network_input = np.load(filepath)
with open('/content/gdrive/My Drive/Google Colab/MusicGen/Data/Mozart/net_out_durations_CLBM_L10E_160', 'rb') as filepath:
  network_output = np.load(filepath)

In [ ]:
print(network_output.shape)
print(network_input.shape)

(165514, 69)
(165514, 160, 2)


In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Activation, Flatten, Conv1D
from keras import utils
from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization as BatchNorm

In [ ]:
num_classes = len(network_output[0])

def create_model(network_input, num_classes):
  model = Sequential()
  model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
  model.add(Dropout(0.4))
  model.add(LSTM(512, return_sequences=True))
  model.add(Dropout(0.4))
  model.add(Flatten())
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
  return model


In [ ]:
filepath = "/content/gdrive/My Drive/Google Colab/MusicGen/DeepRhythm_Checkpoints/BM/Ver_1/deepRhythm-{epoch:02d}-{loss:.4f}.hdf5"

BATCH_SIZE = 64
STEPS_PER_EPOCH = len(network_output) / BATCH_SIZE
SAVE_PERIOD = 5
EPOCHS = 70
SAVE_EACH_PERIOD = True
SAVE_FREQ = int(SAVE_PERIOD * STEPS_PER_EPOCH)

checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=False, 
    save_weights_only=False,
    mode='min',
    save_freq=SAVE_FREQ #'epoch'
)
callbacks_list = [checkpoint]

In [ ]:
from matplotlib import pyplot

model = create_model(network_input, num_classes) 
model_history = model.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks_list)
  
pyplot.plot(model_history.history['loss'], label='train')
pyplot.legend()
pyplot.show()